In [10]:
%load_ext autoreload
%autoreload 1
%aimport ecg_get_data
%aimport Models
%aimport train_test_validat
%aimport self_attention
%aimport ECGplot
%aimport Net
%aimport select_dataset
import select_dataset
import Models 
import Net
from train_test_validat import *
from self_attention import *
import  ecg_get_data 
import matplotlib.pyplot as plt
import ecg_plot

import torch
import torch.utils.data as Data
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import random

import time
import os
import gc

random_seed = 2
torch.manual_seed(random_seed)    # reproducible
torch.cuda.manual_seed_all(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)

time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime()) 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
model_path = './model/'+time_str
log_path = './log/'+  time_str
ECG_root = '/workspace/data/Preprocess_HTN/data/ECG'
EcgChannles_num = 12
EcgLength_num = 5000
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


In [12]:
data = select_dataset.splite_dataset('/workspace/data/Preprocess_HTN/data/',True)

In [13]:
test_list = data.__get_test_file_list__(False)
test_Dataset = ecg_get_data.ECG_Dataset(ECG_root,test_list,EcgChannles_num,EcgLength_num)
# print(valid_list)
valid_list,train_list,addition_train_list = data.__get_VT_file_list__(0.1,True)
valid_Dataset = ecg_get_data.ECG_Dataset(ECG_root,valid_list,EcgChannles_num,EcgLength_num)
train_Dataset = ecg_get_data.ECG_Dataset(ECG_root,train_list,EcgChannles_num,EcgLength_num,ECG_root,addition_train_list)

	
 orginal   fliteryears
  33342       4347   
	
 orginal    fliterID 
   509        425    
	
 orginal    fliterID 
   4347       3807   
	
       HTN  NHTN 
test   425  3807 
npys:{%d} 4232
	
 orginal   fliteryears
  65933      14254   
	
 orginal    fliterID 
   1354       1221   
	
 orginal    fliterID 
  14254      13439   
	
       HTN  NHTN 
train 1099  1099 
valid  122   122 
 add    0   12218
npys:{%d} 244
npys:{%d} 2198
shadow_npys:{%d} 12218


In [14]:
ECG,label = test_Dataset.__getitem__(1)
#inf,path = train_Dataset.get_basic_inf(55)
#ecg_plot.plot(ECG*3500/1000, sample_rate = 500, title = "test",row_height= 10,show_grid=True,show_separate_line=True)
label
#ecg_plot.save_as_png(inf[1],'/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN//PNG_ECG/',dpi = 100)

tensor(1)

In [15]:
BATCH_SIZE = 128

FOLDS = 1
EPOCHS = 5000  
PATIENCE = 100
LR = 0.01


In [16]:
torch.cuda.empty_cache()# 清空显卡cuda
# NET = [
#     Net.channels_branch_CNN(True,res = True,se = True,Dropout_rate = 0.25)
# ]
# NET = [Net.channels_branch_CNN(True,res = True,se = True,Dropout_rate = 0.25) ] # type: ignore
NET = [Net.MLBFNet(True,res = True,se = True,Dropout_rate = 0.25) ] # type: ignore

In [17]:
from torch.utils.tensorboard import SummaryWriter
os.makedirs(model_path, exist_ok=True)
writer = SummaryWriter(log_path)
# writer.add_graph(NET[0], torch.zeros((1,12,5000)))  #模型及模型输入数据
torch.cuda.empty_cache()# 清空显卡cuda

In [18]:
import math
for fold in range(FOLDS):
    #每个人fold都重新抽取
    NET[fold].to(DEVICE)
    valid_list,train_list,addition_train_list = data.__get_VT_file_list__(0.2,True)
    valid_Dataset = ecg_get_data.ECG_Dataset(ECG_root,valid_list,EcgChannles_num,EcgLength_num)
    train_Dataset = ecg_get_data.ECG_Dataset(ECG_root,train_list,EcgChannles_num,EcgLength_num,ECG_root,addition_train_list)
    
    train_dataloader = Data.DataLoader(dataset=train_Dataset, batch_size=BATCH_SIZE, shuffle=True,num_workers=16,pin_memory=True)
    valid_dataloader = Data.DataLoader(dataset=valid_Dataset, batch_size=BATCH_SIZE, shuffle=True,num_workers=16,pin_memory=True)
    test_dataloader = Data.DataLoader(dataset=test_Dataset, batch_size=25, shuffle=True,num_workers=1,pin_memory=True)
    early_stopping = EarlyStopping(PATIENCE, verbose=True, model_path=model_path, delta=0, positive=False)
    optimizer  = torch.optim.Adam(NET[fold].parameters(), lr=LR,weight_decay=1e-2)  
    
    warm_up_iter = 6
    T_max = 16	# 周期
    lr_max = 1e-3	# 最大值
    lr_min = 1e-5	# 最小值
    lambda0 = lambda cur_iter: (cur_iter / warm_up_iter)*lr_max if  cur_iter < warm_up_iter else \
        (lr_min + 0.5*(lr_max-lr_min)*(1.0+math.cos( (cur_iter-warm_up_iter)/(T_max-warm_up_iter)*math.pi)))/0.1
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda0)
    criterion = torch.nn.CrossEntropyLoss()   

    for epoch in range(1,EPOCHS):
        time_all=0
        start_time = time.time()
        y_true,y_pred,train_loss,train_acc = train_model(train_dataloader, NET[fold], criterion, optimizer,DEVICE) # type: ignore # 训练模型
        time_all = time.time()-start_time
        C0 = confusion_matrix(y_true,y_pred)
        F1_score_train =f1_score(y_true, y_pred, average='macro')#F1分数
        y_true,y_pred,validate_loss,validate_acc = eval_model(valid_dataloader,criterion,NET[fold],DEVICE) # 验证模型
        F1_score_valid =f1_score(y_true, y_pred, average='macro')#F1分数
        writer.add_scalars(main_tag=str(fold)+'_Loss',tag_scalar_dict={'train': train_loss,'validate': validate_loss},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_Accuracy',tag_scalar_dict={'train': train_acc,'validate': validate_acc},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_LearningRate',tag_scalar_dict={'LR': optimizer.state_dict()['param_groups'][0]['lr']},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_F1_score',tag_scalar_dict={'train':F1_score_train,'validate': F1_score_valid},global_step=epoch)
        print('- Epoch: %d - Train_loss: %.5f - Train_acc: %.5f - F1 score: %.5f - Val_loss: %.5f - Val_acc: %.5f - F1 score: %.5f - T_Time: %.5f' %(epoch,train_loss,train_acc,F1_score_train,validate_loss,validate_acc,F1_score_valid,time_all))
        print('当前学习率：%.8f' %optimizer.state_dict()['param_groups'][0]['lr'])
        C1 = confusion_matrix(y_true,y_pred)
        print('train:\n',C0)
        print('validate:\n',C1)
        
        scheduler.step() # 学习率迭代
        #是否满足早停法条件
        if(early_stopping(validate_loss,NET[fold],fold)):
            print("Early stopping")
            break

    print('Fold %d Training Finished' %(fold+1))
    torch.cuda.empty_cache()# 清空显卡cuda
print('Training Finished')

	
 orginal   fliteryears
  65933      14254   
	
 orginal    fliterID 
   1354       1221   
	
 orginal    fliterID 
  14254      13439   
	
       HTN  NHTN 
train 3175   977 
valid  488   244 
 add    0   24436
npys:{%d} 732
npys:{%d} 4152
shadow_npys:{%d} 24436
- Epoch: 1 - Train_loss: 0.69418 - Train_acc: 0.49855 - F1 score: 0.49235 - Val_loss: 0.69318 - Val_acc: 0.50159 - F1 score: 0.33938 - T_Time: 68.58494
当前学习率：0.00000000
train:
 [[ 822 1254]
 [ 832 1244]]
validate:
 [[  2 364]
 [  0 366]]
Validation F1 score  increase to (inf --> -0.69317944).  Saving model ...
                    --------------------------------------------------

- Epoch: 2 - Train_loss: 0.69397 - Train_acc: 0.50277 - F1 score: 0.49767 - Val_loss: 0.68917 - Val_acc: 0.51744 - F1 score: 0.44502 - T_Time: 62.31663
当前学习率：0.00000167
train:
 [[ 872 1204]
 [ 868 1208]]
validate:
 [[ 57 309]
 [ 44 322]]
Validation F1 score  increase to (-0.69317944 --> -0.68916990).  Saving model ...
                    -----------